# 2024 Guide Session 4주차 실습

In [4]:
'''1'''
# 필요한 라이브러리를 불러오기
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Jupyter Notebook에서 그래프가 바로 보이도록 설정
%matplotlib inline

# ignore warnings - 필요없는 경고 메시지 표시 X
import warnings
warnings.filterwarnings('ignore')

In [3]:
'''2'''
# 데이터 파일을 불러오기
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
'''3'''
# train data > 칼럼별로 결측치 비율 확인
for col in df_train.columns:
    msg = 'column: {:>10}\t percent of NaN value: {:.2f}%'.format(col, 100 * (df_train[col].isnull().sum() / df_train[col].shape[0]))
    print(msg)

In [ ]:
'''4'''
# test data > 칼럼별로 결측치 비율 확인
for col in df_test.columns:
    msg = 'column: {:>10}\t percent of NaN value: {:.2f}%'.format(col, 100 * (df_test[col].isnull().sum() / df_test[col].shape[0]))
    print(msg)

<span style = "color:red"> 
Q1. 결측치가 있는 칼럼 중, 삭제하는 방식으로 결측치를 처리해야 할 칼럼은 어떤 것인가요?

In [ ]:
'''5'''
### 답
# Cabin

## Column별 전처리 및 시각화

### - 4주차 column: Pclass / Age / Embarked / Fare / Ticket / Cabin

#### 1. Pclass: 티켓 클래스 (1 = 1st, 2 = 2nd, 3 = 3rd)

In [ ]:
'''6'''
# Pclass는 순서형, 카테고리 형 데이터 타입이다.
df_train['Pclass'].value_counts()

In [ ]:
'''7'''
# Pclass 별 생존자의 인원 분포
df_train[['Pclass', 'Survived']].groupby(['Pclass']).sum()

In [ ]:
'''8'''
# 탑승자 분포 확인 
fig, ax = plt.subplots(1, 1, figsize=(8, 6))  # 1행 1열의 서브플롯 설정
bar1 = df_train[['Pclass', 'Survived']].groupby(['Pclass']).count().plot.bar(ax=ax)
bar1.set_title('Pclass - Occupant')
plt.xticks(rotation=0)
plt.show()

In [ ]:
'''9'''
# pclass 별 생존자
bar2 = sns.countplot(hue='Survived', x='Pclass', data=df_train)
bar2.set_title('Pclass - Survived')
plt.show()

In [ ]:
'''10'''
fig, ax = plt.subplots(1,2,figsize=(10,4))
 
# Survived 가 0인 데이터를 이용한 Pie Plot
df_train[df_train['Survived'] == 0]['Pclass'].value_counts().sort_index().plot.pie(ax=ax[0], autopct='%1.1f%%')
ax[0].set(ylabel='', title='Survived=0 - Pclass')
 
# Survived 가 1인 데이터를 이용한 Pie Plot
df_train[df_train['Survived'] == 1]['Pclass'].value_counts().sort_index().plot.pie(ax=ax[1], autopct='%1.1f%%')
ax[1].set(ylabel='', title='Survived=1 - Pclass')
 
plt.show()

In [ ]:
'''11'''
# 생존율
survival_rate = df_train.groupby('Pclass')['Survived'].mean().reset_index()  
survival_rate

In [ ]:
'''12'''
# 객실 등급(Pclass)에 따른 생존율
plt.figure(figsize=(10, 6))
sns.barplot(x='Pclass', y='Survived', data=survival_rate)
plt.title('Survival Rate by Pclass')
plt.ylabel('Survival Rate')
plt.xlabel('Pclass')
plt.show()

<span style = "color:red"> 
Q2. Pclass와 생존율 사이에 이러한 관계가 나타나는 이유를 추론해보세요.

In [ ]:
'''13'''
### 답
# (위치, 구조 순서, 안전 시설 등의 차이)

#### 2. Age: 나이(세)
#### - train data, test data 결측치 有

In [ ]:
'''14'''
df_train['Age'].describe()

In [ ]:
'''15'''
# 결측치 처리 (수치형 데이터)

# 평균으로 대체
df_train['Age'].fillna(df_train['Age'].mean(), inplace=True)
df_test['Age'].fillna(df_test['Age'].mean(), inplace=True)

# (2) 중앙값으로 대체
# df_train['Age'].fillna(df_train['Age'].median(), inplace=True)
# df_test['Age'].fillna(df_test['Age'].median(), inplace=True)

In [ ]:
'''16'''
# 정수형으로 변환
df_train["Age"] = df_train["Age"].astype(int) 
df_test[ "Age"] = df_test["Age"].astype(int)

<span style = "color:red"> 
Q3. Age 칼럼에 정수형 변환을 적용하는 이유는 무엇일까요?

In [ ]:
'''17'''
### 답
# 나이를 소수로 표현할 이유가 없기 때문(측정의 편리성을 위해)

In [ ]:
'''18'''
# Age histogram
fig, ax = plt.subplots(figsize=(10,6))
sns.histplot(df_train['Age'], bins=25, ax=ax)
plt.show()

In [ ]:
'''19'''
# Age boxplot
fig, ax = plt.subplots(figsize=(10, 6))
sns.boxplot(x=df_train['Age'], ax=ax)
ax.set_title('Age Boxplot')
plt.show()

In [ ]:
'''20'''
# Age(나이)에 따른 생존율

# 생존율을 입력받을 list 생성
age_range_survival_ratio = []

# 1살부터 생존율 구하기
for i in range(1, 80):  # 1살부터 79살까지 반복

    # i살 미만의 사람들을 필터링
    age_group = df_train[df_train['Age'] < i]
    
    # 해당 나이대의 생존자 수와 전체 인원 수를 구합니다.
    survived_count = age_group['Survived'].sum()
    total_count = len(age_group)
    
    # 생존율 계산 (0으로 나누는 것을 방지하기 위해 조건 추가)
    if total_count > 0:
        survival_ratio = survived_count / total_count  # 전체 인원 중 생존 인원의 비율
    else:
        survival_ratio = 0
    
    # 리스트에 생존율 추가
    age_range_survival_ratio.append(survival_ratio)


In [ ]:
'''21'''
plt.figure(figsize=(7,7))
plt.plot(age_range_survival_ratio)
plt.title('Survival Rate by Age')
plt.ylabel('Survival Rate')
plt.xlabel('Age')
 
plt.show()

In [ ]:
'''22'''
# 생존여부에 따른 Age 분포의 KDE
fig, ax = plt.subplots(1, 1, figsize=(9, 5))
sns.kdeplot(df_train[df_train['Survived'] == 1]['Age'], ax=ax)
sns.kdeplot(df_train[df_train['Survived'] == 0]['Age'], ax=ax)
plt.legend(['Survived = 1', 'Survived = 0'])
plt.show()

<span style = "color:red"> 
Q4. 위의 그래프 결과를 바탕으로 생존자와 비생존자의 나이 분포를 비교해 보세요.

In [ ]:
'''23'''
### 답
# 탑승비율에 비해 어린이가 많이 생존함. 20~40대의 사람이 생존율이 낮다. 

#### 3. Embarked: 탑승 항구
#### - train data 결측치 有 

In [ ]:
'''24'''
df_train['Embarked'].value_counts()

In [ ]:
'''25'''
# 결측치 처리 (범주형 데이터)
# 범주형 데이터라서 평균, 중앙값 활용 X
# 최빈값으로 대체
most_frequent_embarked = df_train['Embarked'].mode()[0]
df_train['Embarked'].fillna(most_frequent_embarked, inplace=True)

In [ ]:
'''26'''
# Embarked의 데이터 분포
fig, ax = plt.subplots(1,2,figsize=(14,5))
df_train['Embarked'].value_counts().plot.pie(ax=ax[0], autopct='%1.1f%%')
ax[0].set(title='Embarked', ylabel='')
sns.countplot(data=df_train, x='Embarked', ax=ax[1])
plt.show()

In [ ]:
'''27'''
fig, ax = plt.subplots(figsize=(10,6))
 
sns.countplot(data=df_train, x='Embarked', hue='Survived', ax=ax)
 
labels=['Survived=0', 'Survived=1']
ax.legend(labels=labels)
 
plt.show()

In [ ]:
'''28'''
# Embarked에 따른 생존율
survival_rate = df_train.groupby('Embarked')['Survived'].mean().reset_index()  
survival_rate

In [ ]:
'''29'''
# 항구 종류에 따른 생존율
sns.barplot(x='Embarked', y='Survived', data=df_train, ci=None)
plt.title('Survival Rate by Embarked')
plt.xlabel('Embarked')
plt.ylabel('Survival Rate')
plt.show()

In [ ]:
'''30'''
# Pclass 별로 데이터 분리 
Pclass1 = df_train[df_train['Pclass']==1]['Embarked'].value_counts()
Pclass2 = df_train[df_train['Pclass']==2]['Embarked'].value_counts()
Pclass3 = df_train[df_train['Pclass']==3]['Embarked'].value_counts()

fig, ax = plt.subplots(figsize=(10,7))
df = pd.DataFrame([Pclass1, Pclass2, Pclass3])
df.index = ['1st class','2nd class','3rd class']
df.plot(kind='bar', stacked=True, ax=ax)
 
# xlabel 회전
plt.xticks(rotation=45)
plt.show()

In [ ]:
'''31'''
# SibSp, Parch(동승자 유무)와도 연관이 있을까?
fig, ax = plt.subplots(1,2,figsize=(13,6))
 
sns.countplot(data=df_train, x='SibSp', hue='Embarked', ax=ax[0])
sns.countplot(data=df_train, x='Parch', hue='Embarked', ax=ax[1])
 
plt.show()

In [ ]:
'''32'''
# 혼자 탄 사람이 Class 3에도 많을까?
fig, ax = plt.subplots(1,2,figsize=(13,6))
sns.countplot(data=df_train, x='SibSp', hue='Pclass', ax=ax[0])
sns.countplot(data=df_train, x='Parch', hue='Pclass', ax=ax[1])
plt.show()

# S항구에서 혼자 Class3에 탄 탑승객의 생존율이 낮다..ㅠㅠ

<span style = "color:red"> 
Q5. 위의 그래프 결과를 바탕으로 Embarked(탑승항구)/Pclass(티켓 클래스)/SibSp, Parch(동승자 유무)를 생존율과 연관 지어 해석해 보세요.

In [ ]:
'''33'''
### 답
# s항구에서 티켓 클래스가 낮고, 동승자가 적을수록 생존율이 낮다. 반대로 c항구에서 티켓 클래스가 높고, 동승자가 있을 경우 생존율이 높았다. 

#### 4. Fare: 탑승 요금 
#### - test data 결측치 有 

In [ ]:
'''34'''
df_train['Fare'].describe()

In [ ]:
'''35'''
# 결측치 처리 (수치형 데이터)

# 중앙값으로 대체
df_test['Fare'].fillna(df_test['Fare'].median(), inplace=True)

In [ ]:
'''36'''
# Fare histogram
fig, ax = plt.subplots(figsize=(10,6))
sns.histplot(df_train['Fare'], bins=25, ax=ax)
plt.show()

In [ ]:
'''37'''
# Fare boxplot # 이상치, 왼쪽부터 min, mean, max.
fig, ax = plt.subplots(figsize=(10, 6))
sns.boxplot(x=df_train['Fare'], ax=ax)
ax.set_title('Fare Boxplot')
plt.show()

In [ ]:
'''38'''
# 이상치 처리 

# 1사분위수(Q1)와 3사분위수(Q3)를 계산
Q1_fare = df_train['Fare'].quantile(0.25)
Q3_fare = df_train['Fare'].quantile(0.75)

# IQR (Interquartile Range) 계산
IQR_fare = Q3_fare - Q1_fare

# 이상치 경계를 설정
lower_bound = Q1_fare - 1.5 * IQR_fare
upper_bound = Q3_fare + 1.5 * IQR_fare

# 이상치를 상한선과 하한선으로 대체
df_train['Fare'] = df_train['Fare'].apply(lambda x: upper_bound if x > upper_bound else (lower_bound if x < lower_bound else x))
df_test['Fare'] = df_test['Fare'].apply(lambda x: upper_bound if x > upper_bound else (lower_bound if x < lower_bound else x))

In [ ]:
'''39'''
# 생존 여부에 따른 Fare 분포의 KDE
# 생존자와 비생존자의 요금 분포를 비교해 볼 수 있다.
fig, ax = plt.subplots(1, 1, figsize=(9, 5))
sns.kdeplot(df_train[df_train['Survived'] == 1]['Fare'], ax=ax)
sns.kdeplot(df_train[df_train['Survived'] == 0]['Fare'], ax=ax)
plt.legend(['Survived = 1', 'Survived = 0'])
plt.show()

In [ ]:
'''40'''
# Pclass(등급)별 Fare(요금) 평균 확인
train_fare_means = df_train.groupby('Pclass')['Fare'].mean().reset_index()

# 시각화
plt.figure(figsize=(10, 6))
sns.barplot(x='Pclass', y='Fare', data=train_fare_means)
plt.title('Average Fare by Pclass')
plt.xlabel('Pclass')
plt.ylabel('Average Fare')
plt.show()

#### 5. Cabin: 객실 넘버
#### - train data, test data 결측치 有 

In [ ]:
'''41'''
# 결측치 비율이 70% 이상이므로 칼럼 제거
df_train = df_train.drop(columns=['Cabin'])
df_test = df_test.drop(columns=['Cabin'])

#### 6. Ticket: 티켓 넘버

In [ ]:
'''42'''
df_train['Ticket'].unique()

In [ ]:
'''43'''
df_train['Ticket'].value_counts()

<span style = "color:red"> 
Q6. 위의 결과를 바탕으로 데이터셋에서 어떤 사실을 알 수 있는지 고르시오.<br>
<span style = "color:black"> 
(1) 모든 승객은 서로 다른 티켓을 소지하고 있다.<br>
(2) 동일한 티켓을 소지한 승객이 존재한다.<br>
(3) 모든 티켓은 한 번씩만 출현한다.<br>
(4) 티켓 번호는 승객의 나이를 나타낸다.
</span>

In [ ]:
'''44'''
### 답
# (2)

In [ ]:
'''45'''
df_train.to_csv('df_train_4.csv', index = False)
df_test.to_csv('df_test_4.csv', index = False)